# PVR Result Dataset and it's Prediction

In [22]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [23]:
rvp = pd.read_csv('rvp_cleaned.csv')
print('Create regressors', flush=True)

Create regressors


In [24]:
def create_regressor(rvp_):
    X = rvp_['PERCENTILE'].values.reshape(-1, 1)
    y = rvp_['RANK'].values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    return regressor

In [25]:
categories = ['GEN', 'EWS', 'SC', 'ST', 'OBC-NCL']
regressors = {
    category: [
        create_regressor(rvp[rvp['CATEGORY'] == category]),
        create_regressor(rvp[rvp['CATEGORY'] == category + '-PwD'])
    ] for category in categories
}

In [26]:
def pvr(perc, pwd, category):
    x = pd.Series([perc])
    z = regressors[category][pwd == 'YES'].predict(x.values.reshape(-1, 1))
    k = float(np.round(z))
    if k <= 0:
        k = 13
    return k

In [27]:
result = pvr(78.2, 'NO', 'GEN')

In [28]:
result

205872.0

In [20]:
type(result)

float

# Prediction for College 

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
dataset = pd.read_csv('cleaned.csv')

In [ ]:
dataset

In [16]:
def finalList(mainrank, perc, category, state, gender, pwd, limit, advrank):
    dataset = pd.read_csv('cleaned.csv')

    dataset["Closing Rank"] = pd.to_numeric(dataset["Closing Rank"], errors="coerce").astype("Int64")
    # The algorithm showed some anomaly when %tile was 100.
    # Hence, the following condition.
    p_adv = pd.DataFrame()
    p_mains = pd.DataFrame()
    if mainrank == '-1':
        rank = float(pvr(perc, pwd, category))
    else:
        rank = mainrank

    if pwd == 'YES':
        if gender == 'M':
            catg = category + '-PwD'
            if rank > 0:
                p_mains = dataset[
                    (dataset['Closing Rank'] >= rank) & ((dataset['Category'] == catg) | (dataset['Category'] == 'OPEN-PwD')) & (
                                dataset['Gender'] == 'Gender-Neutral') & (dataset['IIT'] == 0)]
            if advrank > 0:
                p_adv = dataset[
                    (dataset['Closing Rank'] >= advrank) & ((dataset['Category'] == catg) | (dataset['Category'] == 'OPEN-PwD')) & (
                                dataset['Gender'] == 'Gender-Neutral') & (dataset['IIT'] == 1)]
        else:
            catg = category + '-PwD'
            if rank > 0:
                p_mains = dataset[
                    (dataset['Closing Rank'] >= rank) & ((dataset['Category'] == catg) | (dataset['Category'] == 'OPEN-PwD')) & (
                                dataset['IIT'] == 0)]
            if advrank > 0:
                p_adv = dataset[
                    (dataset['Closing Rank'] >= advrank) & ((dataset['Category'] == catg) | (dataset['Category'] == 'OPEN-PwD')) & (
                                dataset['IIT'] == 1)]
    else:
        if gender == 'M':
            if rank > 0:
                p_mains = dataset[
                    (dataset['Closing Rank'] >= rank) & ((dataset['Category'] == category) | (dataset['Category'] == 'OPEN')) & (
                                dataset['Gender'] == 'Gender-Neutral') & (dataset['IIT'] == 0)]
            if advrank > 0:
                p_adv = dataset[
                    (dataset['Closing Rank'] >= advrank) & ((dataset['Category'] == category) | (dataset['Category'] == 'OPEN')) & (
                                dataset['Gender'] == 'Gender-Neutral') & (dataset['IIT'] == 1)]
        else:
            if rank > 0:
                p_mains = dataset[
                    (dataset['Closing Rank'] >= rank) & ((dataset['Category'] == category) | (dataset['Category'] == 'OPEN')) & (
                                dataset['IIT'] == 0)]
            if advrank > 0:
                p_adv = dataset[
                    (dataset['Closing Rank'] >= advrank) & ((dataset['Category'] == category) | (dataset['Category'] == 'OPEN')) & (
                                dataset['IIT'] == 1)]

    if not p_adv.empty and (not p_mains.empty):
        p = pd.concat([p_adv, p_mains])
    elif not p_mains.empty:
        p = p_mains
    else:
        p = p_adv

    v = []
    for i in p.index:
        if p['State'][i] == state:
            if p['Quota'][i] != 'HS':
                v.append(i)
        elif (p['Quota'][i] != 'OS') and (p['Quota'][i] != 'AI'):
            v.append(i)

    q = p.drop(index=v)
    if q.shape[0] > 0:
        q = q.sort_values(by='Closing Rank')
        q = q[0:limit]
        x = q.drop(['Unnamed: 0', 'index', 'Category', 'Opening Rank', 'IIT', 'Round'], axis=1).drop_duplicates()
        x.reset_index(inplace=True, drop=True)
        x.index = x.index + 1
        return x
    else:
        return pd.DataFrame()

In [18]:
# finalList(int(mains_rank), float(percentile), category, home_state, gender, pwd, limit, int(advanced_rank))
result_list = finalList(403, 69.3, 'GEN', 'Rajasthan', 'M', 'NO', 10, 562)

In [19]:
result_list

,Institute,Academic Program Name,Quota,Gender,Closing Rank,Course Time,Degree Type,State
1,"School of Planning & Architecture, Bhopal",Architecture,AI,Gender-Neutral,429,5,Bachelor of Architecture,Madhya Pradesh
2,National Institute of Technology Calicut,Architecture,OS,Gender-Neutral,491,5,Bachelor of Architecture,Kerala
3,Indian Institute of Technology Delhi,Electrical Engineering,AI,Gender-Neutral,574,4,Bachelor of Technology,New Delhi
4,School of Planning & Architecture: Vijayawada,Architecture,AI,Gender-Neutral,581,5,Bachelor of Architecture,Andhra Pradesh
5,Indian Institute of Technology Guwahati,Computer Science and Engineering,AI,Gender-Neutral,589,4,Bachelor of Technology,Assam
6,Indian Institute of Technology Hyderabad,Computer Science and Engineering,AI,Gender-Neutral,608,4,Bachelor of Technology,Telangana
7,"National Institute of Technology, Rourkela",Architecture,OS,Gender-Neutral,622,5,Bachelor of Architecture,Odisha
8,Maulana Azad National Institute of Technology ...,Planning,OS,Gender-Neutral,663,4,Bachelor of Planning,Madhya Pradesh
9,Indian Institute of Technology Delhi,Electrical Engineering (Power and Automation),AI,Gender-Neutral,797,4,Bachelor of Technology,New Delhi
10,"School of Planning & Architecture, Bhopal",Planning,AI,Gender-Neutral,818,4,Bachelor of Planning,Madhya Pradesh


In [21]:
type(result_list)

pandas.core.frame.DataFrame